In [1]:
#import modules
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

pd.options.display.max_columns=999
pd.options.display.max_rows=3000

In [2]:

from openpyxl.workbook import Workbook


In [3]:
df = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

In [4]:
df.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

In [5]:
for i in df.keys():
    print(i)
    print(df[i].head())
    print('\n')
    print('\n')

Loan Details
      Loan_id  Debt_to_burden_Ratio  total_income  TENURE  Sanctioned Amount
0  A999993387              0.259113     5326806.0      36           22000000
1  A999990339              0.323385     8392773.0      48           73000000
2  A999971854              0.346267    12000000.0      48           80000000
3  A999952066              0.229800     9000000.0      36           50000000
4  A999938479              0.370082     8500000.0      36           67000000




Loan_Status_MartoMay
      Loan_id Bucket  TENURE  PRINBALANCE  Months on Books  Sanctioned Amount  \
0  A999993387    TB0      36   13785200.0               18           22000000   
1  A999993387    TB0      36   14361500.0               17           22000000   
2  A999990339    TB0      48   68114900.0                6           73000000   
3  A999990339    TB0      48   68957500.0                5           73000000   
4  A999990339    TB0      48   69805500.0                4           73000000   

   Month  Unn

In [ ]:
# loan_detail = df['Loan Details']
# loan_detail.head()

In [ ]:
# loan_stat_marmay = df['Loan_Status_MartoMay']
# loan_stat_marmay.head()

In [ ]:
# loan_stat_aprjun = df['Loan_Status_AprtoJun']
# loan_stat_aprjun.head()


In [ ]:
# loan_stat_aprjun = loan_stat_aprjun[['Loan_id', 'Bucket', 'TENURE', 'PRINBALANCE', 'Months on Books', 'Sanctioned Amount', 'Month', 'Unnamed: 7', 'Unnamed: 8']]

In [ ]:
# loan_stat_aprjun.head()

In [ ]:
# loan_stat_months = pd.concat([df['Loan_Status_MartoMay'], df['Loan_Status_AprtoJun']])

In [ ]:
# print(loan_stat_months.head())
# print(loan_stat_months.tail())

In [6]:
mar_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'March']
mar_hist_data = df['Historical 6 Months Details'].loc[df['Historical 6 Months Details']['Month'] == 'March']

In [7]:
mar_data_1 = pd.merge(mar_data, mar_hist_data, how='left', on='Loan_id', suffixes=('_mar', '_mar'))

In [8]:
mar_data_1.head()

,Loan_id,Bucket,TENURE,PRINBALANCE,Months on Books,Sanctioned Amount,Month_mar,Unnamed: 7,Unnamed: 8,paidcure,paiduncure,unpaid,rollb,rollf,num6mosdel,num3mosdel,num6mosdel_2plus,num3mosdel_2plus,max6del,max3del,Month_mar
0,A999993387,TB0,36,14361500.0,17,22000000,March,NaN,576300.0,6,0,0,0,0,0,0,0,0,0,0,March
1,A999990339,TB0,48,69805500.0,4,73000000,March,NaN,848000.0,4,0,0,0,0,0,0,0,0,0,0,March
2,A999952066,TB0,36,37728800.0,12,50000000,March,NaN,1237600.0,6,0,0,0,0,0,0,0,0,0,0,March
3,A999937745,TB0,36,45753500.0,3,48000000,March,NaN,NaN,3,0,1,0,0,0,0,0,0,0,0,March
4,A999873098,TB0,48,42680200.0,12,50000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March


In [36]:
mar_data_1[mar_data_1['paidcure'].isnull()]

,Loan_id,Bucket,TENURE,PRINBALANCE,Months on Books,Sanctioned Amount,Month_mar,Unnamed: 7,Unnamed: 8,paidcure,paiduncure,unpaid,rollb,rollf,num6mosdel,num3mosdel,num6mosdel_2plus,num3mosdel_2plus,max6del,max3del,Month_mar


In [ ]:
# bucket_data = mar_data[['Loan_id', 'Bucket']]

In [ ]:
# bucket_data.columns = ['Loan_id', 'mar_bucket'] 

In [ ]:
# bucket_data.head()

In [ ]:
# apr_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'April']

In [ ]:
# apr_data.head()

In [9]:
jun_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'June']

In [18]:
len(jun_data)

29115

In [10]:
jun_data.head()

,Loan_id,Bucket,PRINBALANCE,TENURE,Months on Books,Sanctioned Amount,Month,Unnamed: 7,Unnamed: 8
2,A999990339,REGULAR,66303700.0,48,7,73000000,June,NaN,NaN
5,A999971854,REGULAR,48509900.0,48,25,80000000,June,NaN,-19605000.0
7,A999952066,REGULAR,32791800.0,36,15,50000000,June,NaN,NaN
10,A999938479,REGULAR,60348600.0,36,5,67000000,June,NaN,NaN
12,A999937745,REGULAR,42353900.0,36,6,48000000,June,NaN,NaN


In [26]:
month_data = pd.merge(mar_data_1, jun_data, how='left', on='Loan_id', suffixes=('_mar', '_jun'))

In [27]:
month_data.head()

,Loan_id,Bucket_mar,TENURE_mar,PRINBALANCE_mar,Months on Books_mar,Sanctioned Amount_mar,Month_mar,Unnamed: 7_mar,Unnamed: 8_mar,paidcure,paiduncure,unpaid,rollb,rollf,num6mosdel,num3mosdel,num6mosdel_2plus,num3mosdel_2plus,max6del,max3del,Month_mar,Bucket_jun,PRINBALANCE_jun,TENURE_jun,Months on Books_jun,Sanctioned Amount_jun,Month,Unnamed: 7_jun,Unnamed: 8_jun
0,A999993387,TB0,36,14361500.0,17,22000000,March,NaN,576300.0,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A999990339,TB0,48,69805500.0,4,73000000,March,NaN,848000.0,4,0,0,0,0,0,0,0,0,0,0,March,REGULAR,66303700.0,48.0,7.0,73000000.0,June,NaN,NaN
2,A999952066,TB0,36,37728800.0,12,50000000,March,NaN,1237600.0,6,0,0,0,0,0,0,0,0,0,0,March,REGULAR,32791800.0,36.0,15.0,50000000.0,June,NaN,NaN
3,A999937745,TB0,36,45753500.0,3,48000000,March,NaN,NaN,3,0,1,0,0,0,0,0,0,0,0,March,REGULAR,42353900.0,36.0,6.0,48000000.0,June,NaN,NaN
4,A999873098,TB0,48,42680200.0,12,50000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,REGULAR,39679600.0,48.0,15.0,50000000.0,June,NaN,NaN


In [28]:
month_data['Bucket_jun'].value_counts()

REGULAR    15847
TB0         1082
TB1          270
Name: Bucket_jun, dtype: int64

In [29]:
def target(row):
    if row['Bucket_jun'] == 'TB1':
        return 1
    else:
        return 0

month_data['target'] = month_data.apply(target, axis=1)

In [30]:
month_data['target'].value_counts()

0    24305
1      270
Name: target, dtype: int64

In [31]:
dups = len(month_data) - len(month_data.drop_duplicates())
month_data.info()
print('\n')
print("Number of duplicates in data frame: " + str(dups))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24575 entries, 0 to 24574
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Loan_id                24575 non-null  object 
 1   Bucket_mar             24575 non-null  object 
 2   TENURE_mar             24575 non-null  int64  
 3   PRINBALANCE_mar        24575 non-null  float64
 4   Months on Books_mar    24575 non-null  int64  
 5   Sanctioned Amount_mar  24575 non-null  int64  
 6   Month_mar              24575 non-null  object 
 7   Unnamed: 7_mar         0 non-null      float64
 8   Unnamed: 8_mar         3 non-null      float64
 9   paidcure               24575 non-null  int64  
 10  paiduncure             24575 non-null  int64  
 11  unpaid                 24575 non-null  int64  
 12  rollb                  24575 non-null  int64  
 13  rollf                  24575 non-null  int64  
 14  num6mosdel             24575 non-null  int64  
 15  nu

In [19]:
month_data[month_data['Bucket_jun'].isnull()]

,Loan_id,Bucket_mar,TENURE_mar,PRINBALANCE_mar,Months on Books_mar,Sanctioned Amount_mar,Month_mar,Unnamed: 7_mar,Unnamed: 8_mar,paidcure,paiduncure,unpaid,rollb,rollf,num6mosdel,num3mosdel,num6mosdel_2plus,num3mosdel_2plus,max6del,max3del,Month_mar,Bucket_jun,PRINBALANCE_jun,TENURE_jun,Months on Books_jun,Sanctioned Amount_jun,Month,Unnamed: 7_jun,Unnamed: 8_jun,target
0,A999993387,TB0,36,14361500.0,17,22000000,March,NaN,576300.0,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
13,A999643203,TB0,36,3226900.0,32,21000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19,A999349812,TB0,48,21203300.0,11,24000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
26,A999000974,TB0,48,35354900.0,9,39000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
30,A998901950,TB0,48,27022100.0,26,45000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24564,A100525553,TB0,48,93344800.0,16,120000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
24566,A100463209,TB0,48,13199200.0,41,70000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
24567,A100410370,TB0,36,48000000.0,0,48000000,March,NaN,NaN,0,0,0,0,0,1,1,0,0,1,1,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
24571,A100080302,TB0,48,26520900.0,16,33000000,March,NaN,NaN,6,0,0,0,0,0,0,0,0,0,0,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [21]:
import numpy as np

In [25]:
month_data.loc[(month_data['Bucket_jun'].isnull()), 'Bucket_jun']='REGULAR'
# month_data['Bucket_jun'].replace('NaN', 'REGULAR', inplace=True)
month_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24575 entries, 0 to 24574
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Loan_id                24575 non-null  object 
 1   Bucket_mar             24575 non-null  object 
 2   TENURE_mar             24575 non-null  int64  
 3   PRINBALANCE_mar        24575 non-null  float64
 4   Months on Books_mar    24575 non-null  int64  
 5   Sanctioned Amount_mar  24575 non-null  int64  
 6   Month_mar              24575 non-null  object 
 7   Unnamed: 7_mar         0 non-null      float64
 8   Unnamed: 8_mar         3 non-null      float64
 9   paidcure               24575 non-null  int64  
 10  paiduncure             24575 non-null  int64  
 11  unpaid                 24575 non-null  int64  
 12  rollb                  24575 non-null  int64  
 13  rollf                  24575 non-null  int64  
 14  num6mosdel             24575 non-null  int64  
 15  nu

In [ ]:
#split dataframe
train, test = train_test_split(month_data, train_size=0.7, random_state=99)

In [ ]:
x_train = train.drop(['target'], axis=1)
y_train = train['target']

x_test = test.drop(['target'], axis=1)
y_test = test['target']

In [ ]:
x_train.head()

In [ ]:
bucket_mar_pivot = train.pivot_table(index='Bucket_jun', values='target')
bucket_mar_pivot.plot.bar()

In [ ]:
# df.keys()

In [ ]:
# df['Call Details'].head()

In [ ]:
# import numpy as np

In [ ]:
# df['Call Details'].pivot_table(index='Application_Id', values=['Right_Party_Contact', 'Promise_to_pay', 'total_contacts'], aggfunc= np.sum)

In [ ]:
# df.keys()

In [ ]:
# data = pd.merge(df['Loan Details'], loan_stat_months, how='left', on='Loan_id')

In [ ]:
# data.head()

In [ ]:
# data = data.drop(columns=['Unnamed: 7', 'Unnamed: 8'], axis=1)
# data.head()

In [ ]:
# hist_data = df['Historical 6 Months Details'].head()
# hist_data.head()

In [ ]:
# data = pd.merge(data, hist_data, how='left', on=['Loan_id', 'Month'])
# data.head()

In [ ]:
# loan_id_map = df['Loan_ID mapping']
# loan_id_map.head()

In [ ]:
# call_details = df['Call Details']
# call_details.head()

In [ ]:
# call_details['month'].value_counts()

In [ ]:
# m = {3:'March', 4:'April', 5:'May'}
# call_details['month'] = call_details['month'].map(m)

In [ ]:
# call_details.head()

In [ ]:
# call_details = pd.merge(loan_id_map, call_details, how='left', left_on='Application_id', right_on='Application_Id')
# call_details.head()

In [ ]:
# data = pd.merge(data, call_details, how='left', left_on=['Loan_id', 'Month'], right_on=['Loanid', 'month'])

In [ ]:
# data.head()


In [ ]:
# writer = pd.ExcelWriter('data.xlsx')
# data.to_excel(writer,'Sheet1')
# writer.save()


In [ ]:
# data.head()
